In [213]:
import pandas as pd
pd.set_option("display.precision", 2)

#

# 1. Read transactions history

In [214]:
file_transactions = 'transactions.csv'
df_trans = pd.read_csv(file_transactions)
df_trans.tail()

,Date,Ticker,Action,Quantity,Price
0,2022-01-13,ASML,Buy,20,730
1,2022-01-13,AAL,Buy,100,19
2,2022-01-14,ASML,Sell,5,740


# Create Positions

In [215]:
df = df_trans
df.sort_values(['Ticker','Date','Action'], ascending=[True, True, True], inplace=True)
df

,Date,Ticker,Action,Quantity,Price
1,2022-01-13,AAL,Buy,100,19
0,2022-01-13,ASML,Buy,20,730
2,2022-01-14,ASML,Sell,5,740


In [216]:
df['Net Amount'] = df.Quantity * df.Price

df['Signed Quantity'] = df.apply(lambda x: ((x.Action == "Buy") - (x.Action == "Sell")) * x['Quantity'], axis = 1)
df['Signed Net Amount'] = df.apply(lambda x: ((x.Action == "Buy") - (x.Action == "Sell")) * x['Net Amount'], axis = 1)
df

,Date,Ticker,Action,Quantity,Price,Net Amount,Signed Quantity,Signed Net Amount
1,2022-01-13,AAL,Buy,100,19,1900,100,1900
0,2022-01-13,ASML,Buy,20,730,14600,20,14600
2,2022-01-14,ASML,Sell,5,740,3700,-5,-3700


In [217]:
df_sum = df.groupby('Ticker')[['Signed Quantity', 'Signed Net Amount']].sum().reset_index()
df_sum

,Ticker,Signed Quantity,Signed Net Amount
0,AAL,100,1900
1,ASML,15,10900


In [218]:

df_sum['Average Price'] = df_sum['Signed Net Amount'] / df_sum['Signed Quantity']
df_sum

,Ticker,Signed Quantity,Signed Net Amount,Average Price
0,AAL,100,1900,19.00
1,ASML,15,10900,726.67
